In [3]:
!pip uninstall -y tensorflow_io

Found existing installation: tensorflow-io 0.31.0
Uninstalling tensorflow-io-0.31.0:
  Successfully uninstalled tensorflow-io-0.31.0


In [4]:
!pip install tensorflow_io

^C


In [3]:
!pip install --upgrade tensorflow_io

ERROR: Could not find a version that satisfies the requirement tensorflow_io (from versions: none)
ERROR: No matching distribution found for tensorflow_io


IMPORTING THE REQUIRED MODULES

In [5]:
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
#import tensorflow_addons as tfa
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow as tf

the default variables for the model

In [7]:
IMAGE_PATH = '..//input//chinese-mnist//data//data//'
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 1
RANDOM_STATE = 2022
TEST_SIZE = 0.2
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 64
NO_EPOCHS = 15
DROPOUT_RATIO = 0.4
PATIENCE = 5
VERBOSE = 1

reading the csv file in the data set

In [8]:
main_data=pd.read_csv('..//input//chinese-mnist//chinese_mnist.csv')

creating a function to automatically return the file name when a row of a dataframe(table) is given

In [9]:
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

.apply function to apply the above to all rows of the file column...actually a new column file is created and all files names will be added..

In [10]:
main_data["file"] = main_data.apply(create_file_name, axis=1)

defining a function to return image array by concatenating default image path with each file name

In [11]:
def read_image_sizes(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    return list(image.shape)

stacking the (width,height) of images in a numpy array and then converting it into a dataframe to concatenating to the original dataframe(main_data) 

In [12]:
m = np.stack(main_data['file'].apply(read_image_sizes))
image_size_df = pd.DataFrame(m,columns=['w','h'])
main_data = pd.concat([main_data,image_size_df],axis=1, sort=False)

Now we CLASSIFY THE CHARACTERS..for this we have to split the dataset into train and  test. further we classify train into train and vaildation . we use RANDOM_STATE and stratify to ensure balanced test sets. stratify actually is used because we are training a model to identify categories.

In [14]:
train_df, test_df = train_test_split(main_data, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=main_data["code"].values)

In [15]:
train_df, val_df = train_test_split(train_df, test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=train_df["code"].values)

The train-validation split is 80% for training set and 20% for validation set.

NOW WE ARE GOING TO BUILD THE MODEL

THIS IS A FUNCTION TO READ IMAGES FROM THEIR FILES QUITE LIKE HOW CV2.IMREAD WORKS AND THEN WE ARE RESIZING THEM USING OUR PREDEFINED WIDTH AND HEIGHT THE CHANNEL IS 1 ENSURING THAT IT IS GRAYSCALE IT FINALLY RETURNS A NUMPY ARRAY

In [16]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT, 1), mode='reflect')
    return image[:,:,:]

THIS FUNCTION IS A BIT COMPLEX ONE.FIRSTLY THE DATASET IS A DATAFRAME.WE ARE READING EACH IMAGE FROM THE DF'S FILE COLUMN AND THEN STACKING IT IN X AND VAR IS ACTUALLY THE NAME OF THE COLUMN THAT WE ARE GONNA ENCODE.get_dummies CREATES A NUMPY ARRAY OF DUMMY VARIABLES.

In [17]:
def categories_encoder(dataset, var='character'):
    X = np.stack(dataset['file'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [18]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

NOW THE MAIN PART OF THE PROBELM WHERE WE DESIGN OUR NEURAL NETWORK BY DEFINING LAYERS AND GIVING CERTAIN TRAINING SIZES

The modes uses 2 Convolutional layers, followed by a MaxPool, a Dropout, then another 2 Convolutional layers and a Dropout. Then follows a Flatten and a Dense layer. We compile the model with Adam optimizer and use a categorical crossentropy loss functions. The metric used is accuracy.

We are using as well a learning function with variable learning rate (depends on the epoch number).

At each training epoch, we evaluate the validation error and, based on its evolution, we decide if we stop the training or continue (with a prededined patience factor - i.e. we only stop if validation is not improving for a certain number of steps (we set the patience to 5 steps). If at a certain step the validation error is improving, we save the current model. We then will load the best model and use it for prediction of test set.

In [19]:
model=Sequential()
model.add(Conv2D(CONV_2D_DIM_1, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS), activation='relu', padding='same'))
model.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(MaxPool2D(MAX_POOL_DIM))
model.add(Dropout(DROPOUT_RATIO))
model.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model.add(Dropout(DROPOUT_RATIO))
model.add(Flatten())
model.add(Dense(y_train.columns.size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

The model consists of a sequence of convolutional layers, max pooling layers, dropout layers, and dense layers.

The convolutional layers extract features from the input image. The Conv2D() layer creates a convolutional layer with CONV_2D_DIM_1 filters of size KERNEL_SIZE. The activation='relu' argument specifies that the ReLU activation function will be used. The padding='same' argument specifies that the output of the convolutional layer will have the same width and height as the input image.
The max pooling layers downsample the feature maps extracted by the convolutional layers. The MaxPool2D() layer creates a max pooling layer with a pool size of MAX_POOL_DIM.
The dropout layers randomly drop out neurons during training, which helps to prevent overfitting. The Dropout() layer with a rate of DROPOUT_RATIO will randomly drop out DROPOUT_RATIO of the neurons in the layer.
The dense layers classify the features extracted by the convolutional layers. The Dense() layer creates a dense layer with y_train.columns.size neurons. The activation='softmax' argument specifies that the softmax activation function will be used, which will output a probability distribution over the classes.
The compile() method configures the model for training. The optimizer='adam' argument specifies that the Adam optimizer will be used. The loss='categorical_crossentropy' argument specifies that the categorical cross-entropy loss function will be used. The metrics=['accuracy'] argument specifies that the accuracy metric will be used to evaluate the model.
Once the model is compiled, it can be trained using the fit() method. The fit() method takes the training data and labels as input and trains the model for a specified number of epochs. The model can then be evaluated on the validation data using the evaluate() method. The evaluate() method returns the loss and accuracy on the validation data. The model can then be used to make predictions on new data using the predict() method.

 FURTHER IMPROVING THE PERFORMANCE
 
 The annealing scheduler reduces the learning rate of the model over time. The LearningRateScheduler() class creates a learning rate scheduler that uses a lambda function to calculate the new learning rate for each epoch. The lambda function takes the epoch number as input and returns a new learning rate. In this case, the new learning rate is calculated as 1e-3 * 0.99 ** (x+NO_EPOCHS), where x is the epoch number and NO_EPOCHS is the number of epochs.
The early stopper stops training if the loss on the validation set does not improve for a specified number of epochs. The EarlyStopping() class creates an early stopper that monitors the loss on the validation set and stops training if the loss does not improve for patience epochs.
The model checkpointer saves the model to a file if the model achieves a new best accuracy on the validation set. The ModelCheckpoint() class creates a model checkpointer that saves the model to a file if the model achieves a new best accuracy on the validation set. The monitor='val_accuracy' argument specifies that the model checkpointer will monitor the accuracy on the validation set. The verbose=VERBOSE argument specifies that the model checkpointer will print a message when the model is saved. The save_best_only=True argument specifies that the model checkpointer will only save the model if it achieves a new best accuracy. The save_weights_only=True argument specifies that the model checkpointer will only save the weights of the model, not the entire model.
These callbacks can be used to improve the performance of the model by preventing overfitting and by saving the model when it achieves a new best accuracy.

In [20]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.99 ** (x+NO_EPOCHS))
earlystopper = EarlyStopping(monitor='loss', patience=PATIENCE, verbose=VERBOSE)
checkpointer = ModelCheckpoint('best_model.h5',
                                monitor='val_accuracy',
                                verbose=VERBOSE,
                                save_best_only=True,
                                save_weights_only=True)

TRAINING THE MODEL


batch_size: The number of samples to process at a time.
epochs: The number of times to iterate over the entire training data.
verbose: The verbosity level, with 0 being silent and 1 being the most verbose.
validation_data: A tuple of (X_val, y_val) data to use for validation.
callbacks: A list of callbacks to use during training.
The callbacks are:

earlystopper: A callback that will stop training early if the validation loss stops improving.
checkpointer: A callback that will save the model to disk at regular intervals.
annealer: A callback that will gradually reduce the learning rate as training progresses.

Here is an explanation of each of the parameters:

batch_size: The batch size is the number of samples that are processed at a time. A larger batch size can improve performance, but it can also use more memory.
epochs: The number of epochs is the number of times that the model will be trained on the entire training data. A larger number of epochs will generally result in a better model, but it will also take longer to train.
verbose: The verbose level controls how much output is generated during training. A value of 0 will suppress all output, while a value of 1 will print out the progress of training.
validation_data: The validation data is a set of data that is not used for training. The model is evaluated on the validation data after each epoch, and this information is used to determine when to stop training.
callbacks: Callbacks are functions that are called during training. They can be used to perform various tasks, such as saving the model to disk, reducing the learning rate, or sending metrics to a monitoring service.

In [21]:
train_model  = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val),
                  callbacks=[earlystopper, checkpointer, annealer])

Epoch 1/15
150/150 [==============================] - ETA: 0s - loss: 1.8667 - accuracy: 0.4263
Epoch 1: val_accuracy improved from -inf to 0.61958, saving model to best_model.h5
150/150 [==============================] - 46s 298ms/step - loss: 1.8667 - accuracy: 0.4263 - val_loss: 1.2979 - val_accuracy: 0.6196 - lr: 8.6006e-04
Epoch 2/15
150/150 [==============================] - ETA: 0s - loss: 0.9474 - accuracy: 0.7029
Epoch 2: val_accuracy improved from 0.61958 to 0.78542, saving model to best_model.h5
150/150 [==============================] - 45s 297ms/step - loss: 0.9474 - accuracy: 0.7029 - val_loss: 0.7082 - val_accuracy: 0.7854 - lr: 8.5146e-04
Epoch 3/15
150/150 [==============================] - ETA: 0s - loss: 0.6009 - accuracy: 0.8087
Epoch 3: val_accuracy improved from 0.78542 to 0.84208, saving model to best_model.h5
150/150 [==============================] - 44s 296ms/step - loss: 0.6009 - accuracy: 0.8087 - val_loss: 0.5127 - val_accuracy: 0.8421 - lr: 8.4294e-04
Epoc

NOW WE OBTAIN THE RESULTS BY EVALUVATING THE MODEL WE GET THE LOSS AND ACCURACY AS A TUPLE

In [22]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('\n'*5)
print(f'Test accuracy% : {score[1]*100:.2f} %')
print(f'Test loss% : {score[0]*100:.2f} %')

Test loss: 0.2088829129934311
Test accuracy: 0.9480000138282776






Test accuracy% : 94.80 %
Test loss% : 20.89 %
